No finetuning => only pretrain model

**Use Dataset(private dataset)**
* sentence-transformers
* pretrain model for s-bert

**READ ME**
* https://www.sbert.net/docs/pretrained_models.html#sentence-embedding-models/
* https://www.sbert.net/docs/pretrained_cross-encoders.html

**USE NOTEBOOKS**
* sentence trainsformers import  : https://www.kaggle.com/code/gloomychan/pppm-s-bert-module-import/notebook
* pretrain models import   :  https://www.kaggle.com/code/gloomychan/pppm-s-bert-pretrained-model-import/notebook?scriptVersionId=90998168

# Import

In [ ]:
import sys
sys.path.append('../input/sentencetrans/sentence-transformers-2.2.0')
from sentence_transformers import SentenceTransformer, util

In [ ]:
import os
import pandas as pd
import numpy as np
import torch

#https://www.sbert.net/examples/applications/semantic-search/README.html
sys.path.append("../input/sentencebert/sentence-transformers")
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('../input/sentence-transformer-models/stsb-roberta-large')

In [ ]:
test_df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv')

In [ ]:
display(test_df.shape)
display(test_df.head())

In [ ]:
# Quick check of description lens (for BERT token len)
import matplotlib.pyplot as plt
df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')

# anchor
anchor_lens = [len(x.split()) for x in df['anchor']]
plt.hist(anchor_lens, bins=30)
plt.show()

In [ ]:
# target
target_lens = [len(x.split()) for x in df['target']]
plt.hist(target_lens, bins=30)
plt.show()

In [ ]:
anchor_txt = test_df["anchor"].values
target_txt = test_df["target"].values

# Predict

In [ ]:
# model.to('cuda')
# going a little longer for user inputed synopsis
model.max_seq_len = 256

In [ ]:
anchors = test_df.anchor.values
targets = test_df.target.values
embedding1 = model.encode(anchors, convert_to_tensor=True)
embedding2 = model.encode(targets, convert_to_tensor=True)

In [ ]:
score_list = []
anchor_list= []
target_list= []
sim_df = pd.DataFrame([])
top_k = min(1, len(embedding1))

# embedding1 = embedding1.to('cuda')
# embedding2 = embedding2.to('cuda')

cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)    
for i in range(len(anchors)):
    anchor_list.append(anchors[i])
    target_list.append(targets[i])
    score_list.append(cosine_scores[i][i].item())

In [ ]:
# score_list

## confirm similarity

In [ ]:
sim_df['anchor'] = anchor_list
sim_df['target'] = target_list
sim_df['score'] = score_list

display(sim_df)

In [ ]:
plt.hist(sim_df['score'], bins=30)
plt.show()

# Submit

In [ ]:
sample_df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/sample_submission.csv')
sample_df.score = score_list

In [ ]:
sample_df.to_csv('submission.csv', index=False)

In [ ]:
sample_df.head()